In [ ]:
%load_ext autoreload
%autoreload 2
# %matplotlib widget
%matplotlib inline

In [ ]:
from collections import Counter
from music21 import stream, instrument, note, chord
import numpy as np
from typing import List, Tuple

In [ ]:
"""
Game of life implementation ripped from:
http://rosettacode.org/wiki/Conway%27s_Game_of_Life#Boardless_approach

Code modified from https://github.com/tompkinsguitar/musicalgameoflife/tree/master
"""

neighboring_cells = [(-1, -1), (-1, 0), (-1, 1), 
                     ( 0, -1),          ( 0, 1), 
                     ( 1, -1), ( 1, 0), ( 1, 1)]
 
def run_life(world, N, space="continuous"):
    """
    Play Conway's game of life for N generations from initial world.

    Returns:
        score: Score
    """
    harmony_seq = []
    melody_seq = []
    for g in range(N + 1):
        display(world, g)
        harmony_notes, melody_notes = world2notes(world, space=space)
        harmony_seq.append(harmony_notes)
        melody_seq.extend(melody_notes)
        counts = Counter(n for c in world for n in offset(neighboring_cells, c))
        world = {c for c in counts 
                if counts[c] == 3 or (counts[c] == 2 and c in world)}
        if len(world) == 0:
            print("World empty, ending early")
            break
    score = create_score(harmony_seq, melody_seq)
    return score


def offset(cells, delta):
    "Slide/offset all the cells by delta, a (dx, dy) vector."
    (dx, dy) = delta
    return {(x+dx, y+dy) for (x, y) in cells}

    
def create_score(harmony_seq: List[chord.Chord], melody_seq: List[note.Note]) -> stream.Score:
    """
    Returns:
        s: Score
    """
    harmony_stream = stream.Stream()
    melody_stream = stream.Stream()
    harmony_stream.append(harmony_seq)
    melody_stream.append(melody_seq)
    s = stream.Score(id="Musical Life")
    p1 = stream.Part(id="Melody")
    p2 = stream.Part(id="Chords")
    p1.append(melody_stream)
    p1.insert(instrument.Piano())
    p2.append(harmony_stream)
    p2.insert(instrument.Piano())
    s.insert(0, p1)
    s.insert(0, p2)
    return s


def world2notes(world, space="continuous") -> Tuple[chord.Chord, List[note.Note]]:
    """
    Returns:
        chord: Chord, melody: List[Note]
    """
    harmony_notes = []
    melody_notes = []
    """pitches are represented as MIDI values; (0,0) = 60 = middle C"""
    for (x, y) in world: #converts the coordinates to a Tonnetz (0, 0) is midi-60
        if space == "continuous":
            n = (x*3 + y*4 + 60)
        elif space == "modular":
            n = ((x*3%12) + (y*4%12) + 60)
        elif space == "fifth":
            n = (x*5 + y*7 + 60)
        elif space == "rotation":
            n = (x*7 + y*4 + 60)
        hnote = note.Note(n)
        # each generation last for a whole note
        hnote.quarterLength = 4.0
        mnote = note.Note(n)
        # evenly fit melody into a whole step
        mnote.quarterLength = 4.0 / len(world)
        harmony_notes.append(hnote)
        melody_notes.append(mnote)
    harmony_chord = chord.Chord(harmony_notes)
    return harmony_chord, melody_notes

 
def display(world, g, xlim=None, ylim=None):
    if xlim is None:
        xlim = (-10, 10)
    if ylim is None:
        ylim = (-10, 10)
    print(f"          GENERATION {g}:")

    Xrange = range(xlim[0], xlim[1] + 1)
    for y in range(ylim[0], ylim[1] + 1):
        print("".join("#" if (x, y) in world else "." for x in Xrange))

In [ ]:
"""a few sample starting points"""
R_pentomino = {(1, 2), (2, 1), (2, 2), (2, 3), (3, 3)}
figure8 = {(1, 5), (1, 6), (2, 3), (2, 5), (2, 6), (3, 2), (4, 5), (6, 1), (6, 2), (7, 1), (7, 2)}
bunnies = {(1, 0), (0, 3), (2, 1), (2, 2), (3, 0), (5, 1), (6, 2), (6, 3), (7, 1)}  # starts small but reproduces quickly
random = {(2, 3), (1, 2), (2, 5), (6, 4), (5, 4), (2, 1), (1, 3), (4, 3)}  # only 8 generations
random2 = {(5, 5), (4, 4), (3, 3), (2, 2), (1, 1), (0, 0)}  # only 2 generations
random3 = {(0, 0), (1, 1), (0, 1), (4, 4), (5, 5), (4, 5)}
blinker = {(1, 0), (1, 1), (1, 2)}  # blinks back and forth every generation
block   = {(0, 0), (1, 1), (0, 1), (1, 0)}  # static (and dissonant)
toad    = {(1, 2), (0, 1), (0, 0), (0, 2), (1, 3), (1, 1)}
glider  = {(0, 1), (1, 0), (0, 0), (0, 2), (2, 1)}  # lower, and lower, and lower...
GosperGliderGun = {(1, 4), (1, 5), (2, 4), (2, 5), (11, 3), (11, 4), (11, 5), (12, 2), (12, 6), (13, 1), (13, 7), (14, 1), (14, 7), (15, 4), (16, 2), (16, 6), (17, 3), (17, 4), (17, 5), (18, 4), (21, 5), (21, 6), (21, 7), (22, 5), (22, 6), (22, 7), (23, 4), (23, 8), (25, 3), (25, 4), (25, 8), (25, 9), (35, 6), (35, 7), (36, 6), (36, 7)}
world   = (block | offset(blinker, (5, 2)) | offset(glider, (15, 5)) | offset(toad, (25, 5))
           | {(18, 2), (19, 2), (20, 2), (21, 2)} | offset(block, (35, 7)))

score = run_life(glider, 10, space="modular") #pick a  starting point and a number of generations
score.write("midi", "out/glider.mid")
# score.show("midi")  # this will play a midi file in default midi player
# score.show()  # this brings up a score but gets really messy with the fast notes and complex beat ratios